In [8]:
import create_population
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction

import pandas as pd


from train_lightgbm_rank_diff import Trainer_lightgbm_rank_diff 
from train_lightgbm_rank_niti import Trainer_lightgbm_rank_niti
from train_lightgbm_time import Trainer_lightgbm_time
from train_lightgbm_rank_kaiki import Trainer_lightgbm_rank_kaiki


from evaluation_lightgbm_rank_diff import Evaluator_lightgbm_rank_diff
from evaluation_lightgbm_rank_niti import Evaluator_lightgbm_rank_niti
from evaluation_lightgbm_time_kaiki import Evaluator_lightgbm_time_kaiki
from evaluation_lightgbm_rank_kaiki import Evaluator_lightgbm_rank_kaiki
from evaluation_pop import Evaluator_pop

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%autoreload

In [2]:
population = create_population.create(from_="2023-12-01", to_="2023-12-31")
pd.set_option('display.max_columns', None)

In [3]:
population

,race_id,date,horse_id
0,202306050101,2023-12-02,2021104145
1,202306050101,2023-12-02,2021107141
2,202306050101,2023-12-02,2021104079
3,202306050101,2023-12-02,2021102752
4,202306050101,2023-12-02,2021103638
...,...,...,...
4069,202309050912,2023-12-28,2017104873
4070,202309050912,2023-12-28,2019100653
4071,202309050912,2023-12-28,2017106240
4072,202309050912,2023-12-28,2018103205


In [3]:
import json
from pathlib import Path

import pandas as pd
import numpy as np
import ast
COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
POPULATION_DIR = Path("..", "data", "00_population")
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
with open(MAPPING_DIR / "place.json", "r") as f:
    place_mapping = json.load(f)


In [13]:
#相対化実験2


In [5]:
population_dir = POPULATION_DIR
populaton_filename = "population.csv"
input_dir = RAWDF_DIR
output_dir= OUTPUT_DIR
input_filename = "horse_results.csv"
output_filename = "horse_results.csv"
race_type_mapping = race_type_mapping
weather_mapping = weather_mapping
ground_state_mapping = ground_state_mapping
race_class_mapping = race_class_mapping
place_mapping = place_mapping

"""
未加工の馬の過去成績テーブルをinput_dirから読み込んで加工し、
output_dirに保存する関数。
"""
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
# df = pd.read_csv(input_dir / input_filename, sep="\t").query(
#     "horse_id in @population['horse_id']"
# )
population_horse_ids = population['horse_id'].tolist()

# クエリでリストを直接使用
df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "horse_id in @population_horse_ids"
)


df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
df.dropna(subset=["rank"], inplace=True)
df["date"] = pd.to_datetime(df["日付"])
df["weather"] = df["天気"].map(weather_mapping)
df["race_type"] = df["距離"].str[0].map(race_type_mapping)
df["course_len"] = df["距離"].str.extract(r"(\d+)").astype(int)
df["ground_state"] = df["馬場"].map(ground_state_mapping)
df["rank_diff"] = df["着差"].map(lambda x: 0 if x < 0 else x)
df["prize"] = df["賞金"].fillna(0)

df["nobori"] = df["上り"].astype(float)

regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["レース名"].str.extract(rf"({regex_race_class})")[0].map(race_class_mapping)
)
df["time"] = pd.to_datetime(df["タイム"], format="%M:%S.%f", errors="coerce")
df["time"] = (
    df["time"].dt.minute * 60
    + df["time"].dt.second
    + df["time"].dt.microsecond / 1000000
)
df["win"] = (df["rank"] == 1).astype(int)
df["rentai"] = (df["rank"] <= 2).astype(int)
df["show"] = (df["rank"] <= 3).astype(int)
df["place"] = df["開催"].str.extract(r"(\D+)")[0].map(place_mapping)
df.dropna(subset=["place"], inplace=True)    
df.rename(columns={"頭数": "n_horses"}, inplace=True)

pase_cols = df['ペース'].str.split('-', expand=True)
pase_cols.columns = [f'pase_{i+1}' for i in range(pase_cols.shape[1])]

# データ型変換と型確認
pase_cols = pase_cols.apply(pd.to_numeric, errors='coerce').astype('float64')

# 元のデータフレームと結合
df = pd.concat([df, pase_cols], axis=1)

# NoneをNaNに置き換え
# df = df.where(pd.notnull(df), np.nan)
df["place"] = df["place"].astype(int)
df["race_class"] = df["race_class"].astype(int)
df["ground_state"] = df["ground_state"].astype(int)
df["weather"] = df["weather"].astype(int)   
# コーナー通過順を分割して列を作成
corner_cols = df['通過'].str.split('-', expand=True)
corner_cols.columns = [f'corner_{i+1}' for i in range(corner_cols.shape[1])]
# オブジェクト型のデータを整数型に変換する
corner_cols = corner_cols.apply(pd.to_numeric, errors='coerce').astype('Int64')  
# nullable int型を指定
# 元のデータフレームと結合
df = pd.concat([df, corner_cols], axis=1)

# NoneをNaNに置き換え
df = df.where(pd.notnull(df), np.nan)

# rank / n_horses の特徴量を作成（欠損値を含む行はNaNに設定）
df["rank_per_horse"] = df["rank"].where(df["rank"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

# corner_1 / n_horses の特徴量を作成（欠損値を含む行はNaNに設定）
df["corner_1_per_horse"] = df["corner_1"].where(df["corner_1"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

df["corner_2_per_horse"] = df["corner_2"].where(df["corner_2"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["corner_3_per_horse"] = df["corner_3"].where(df["corner_3"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["corner_4_per_horse"] = df["corner_4"].where(df["corner_4"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

df["corner_1_per_horse"].head(100)

29864         1.0
29865       0.625
29866         0.9
29867         0.5
29868    0.666667
           ...   
65547    0.181818
65548    0.307692
65549    0.666667
65550         0.6
65551    0.266667
Name: corner_1_per_horse, Length: 100, dtype: Float64

In [10]:
#まとめて実行
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
# レース情報テーブルの前処理
race_info_preprocessed = preprocessing.process_race_info()
# 騎手リーディングテーブルの前処理
jockey_leading_preprocessed = preprocessing.process_jockey_leading()
# 調教師リーディングテーブルの前処理
trainer_leading_preprocessed = preprocessing.process_trainer_leading()
# 血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds()
# 種牡馬リーディングテーブルの前処理
sire_leading_preprocessed = preprocessing.process_sire_leading()

In [30]:
#特徴量バグ修正
from pathlib import Path

from tqdm.notebook import tqdm
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.notebook import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO 

DATA_DIR = Path("..", "data")
POPULATION_DIR = DATA_DIR / "00_population"
INPUT_DIR = DATA_DIR / "01_preprocessed"
OUTPUT_DIR = DATA_DIR / "02_features"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
poplation_filename = "population.csv"
population_dir: Path = POPULATION_DIR
input_dir: Path = INPUT_DIR

population = pd.read_csv(population_dir / poplation_filename, sep="\t")
horse_results_filename = "horse_results.csv"
horse_results = pd.read_csv(input_dir / horse_results_filename, sep="\t")

In [31]:
print(population["race_id"].dtype)
print(horse_results["horse_id"].dtype)

int64
int64


In [46]:
baselog = (
    population.merge(
        horse_results, on="horse_id", suffixes=("", "_horse")
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
print(baselog["horse_id"].dtype)
print(baselog["race_id"].dtype)

int64
int64


In [75]:
merged_df = population.copy()
merged_df

,race_id,date,horse_id
0,202306050101,2023-12-02,2021104145
1,202306050101,2023-12-02,2021107141
2,202306050101,2023-12-02,2021104079
3,202306050101,2023-12-02,2021102752
4,202306050101,2023-12-02,2021103638
...,...,...,...
4069,202309050912,2023-12-28,2017104873
4070,202309050912,2023-12-28,2019100653
4071,202309050912,2023-12-28,2017106240
4072,202309050912,2023-12-28,2018103205


In [79]:
grouped_df = baselog.groupby(["race_id", "horse_id"])
print(grouped_df["race_id"].dtype)
df = (
    grouped_df.head(5)
    .groupby(["race_id", "horse_id"])[["prize"]]
    .mean()
    .add_suffix(f"_{5}races")
)
df

race_id       horse_id  
202306050101  2021102675    int64
              2021102750    int64
              2021102752    int64
              2021102925    int64
              2021103111    int64
                            ...  
202309050912  2019105139    int64
              2019106102    int64
              2020100105    int64
              2020100815    int64
              2020101792    int64
Name: race_id, Length: 3522, dtype: object


prize_5races
race_id      horse_id                
202306050101 2021102675      0.000000
             2021102750      0.000000
             2021102752     86.250000
             2021102925     27.666667
             2021103111      0.000000
...                               ...
202309050912 2019105139     92.800000
             2019106102    149.560000
             2020100105    661.340000
             2020100815    603.100000
             2020101792    586.000000

[3522 rows x 1 columns]

In [65]:
tdf = merged_df.merge(grouped_df, on=["race_id", "horse_id"],how= "left")
tdf

TypeError: Can only merge Series or DataFrame objects, a <class 'pandas.core.groupby.generic.DataFrameGroupBy'> was passed

In [17]:
fc = FeatureCreator()

In [25]:
#race_typeの型が不一致、バグ修正
fc.baselog = (
            fc.population.merge(
                fc.horse_results, on="horse_id", suffixes=("", "_horse")
            )
            .query("date_horse < date")
            .sort_values("date_horse", ascending=False)
        )

merged_df = population.copy()
grouped_df = fc.baselog.groupby(["race_id", "horse_id"])

print(merged_df["race_id"].dtype)
print(grouped_df["race_id"].dtype)


int64
race_id       horse_id  
202306050101  2021102675    int64
              2021102750    int64
              2021102752    int64
              2021102925    int64
              2021103111    int64
                            ...  
202309050912  2019105139    int64
              2019106102    int64
              2020100105    int64
              2020100815    int64
              2020101792    int64
Name: race_id, Length: 3522, dtype: object


In [15]:
#crossのバグ修正
merged_df = fc.population.copy()        
df = (
    fc.results[["race_id", "horse_id", "wakuban", "umaban", "sex"]]
    .merge(fc.race_info[["race_id", "race_type", "around"]], on="race_id")
    .merge(merged_df[["race_id", "horse_id", "date"]], on=["race_id", "horse_id"], how="left")
)


# wakuban と race_type の交互作用特徴量
df["wakuban_race_type"] = df["race_type"].map({0: 1, 1: -1, 2: 0}).fillna(0) * df["wakuban"]
df["wakuban_around"] = df["around"].map({2: 1}).fillna(0) * df["wakuban"]
df["umaban_race_type"] = df["race_type"].map({0: 1, 1: -1, 2: 0}).fillna(0) * df["umaban"]
df["umaban_around"] = df["around"].map({2: 1}).fillna(0) * df["umaban"]


# 季節 (日付) と性別に基づく交互作用特徴量
df["date_1"] = pd.to_datetime(df["date"])
df["sin_date"] = np.sin(2 * np.pi * df["date_1"].dt.dayofyear / 365)
df["cos_date"] = np.cos(2 * np.pi * df["date_1"].dt.dayofyear / 365) + 1

df["sin_date_sex"] = df["sex"].map({0: -1, 1: 1}) * df["sin_date"]
df["cos_date_sex"] = df["sex"].map({0: -1, 1: 1}) * df["cos_date"]


merged_df = merged_df.merge(
    df[["race_id", "horse_id", "date","wakuban_race_type", "wakuban_around","umaban_race_type","umaban_around", "sin_date_sex", "cos_date_sex"]],
    on=["race_id","date", "horse_id"],
    how="left"
)
merged_df = merged_df.astype({col: 'float32' for col in merged_df.select_dtypes('float64').columns})
merged_df = merged_df.astype({col: 'int32' for col in merged_df.select_dtypes('int64').columns})

merged_df

# self.agg_cross_features_df= merged_df


,race_id,date,horse_id,wakuban_race_type,wakuban_around,umaban_race_type,umaban_around,sin_date_sex,cos_date_sex
0,442587189,2023-12-02,2021104145,8.0,0.0,15.0,0.0,0.478734,-1.877960
1,442587189,2023-12-02,2021107141,6.0,0.0,12.0,0.0,-0.478734,1.877960
2,442587189,2023-12-02,2021104079,5.0,0.0,10.0,0.0,0.478734,-1.877960
3,442587189,2023-12-02,2021102752,2.0,0.0,4.0,0.0,0.478734,-1.877960
4,442587189,2023-12-02,2021103638,3.0,0.0,5.0,0.0,0.478734,-1.877960
...,...,...,...,...,...,...,...,...,...
4069,445588000,2023-12-28,2017104873,-6.0,0.0,-11.0,0.0,0.051620,-1.998667
4070,445588000,2023-12-28,2019100653,-8.0,0.0,-15.0,0.0,-0.051620,1.998667
4071,445588000,2023-12-28,2017106240,-4.0,0.0,-7.0,0.0,0.051620,-1.998667
4072,445588000,2023-12-28,2018103205,-8.0,0.0,-16.0,0.0,0.051620,-1.998667


In [11]:
fc = FeatureCreator()
features = fc.create_features()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_jockey()...
running agg_jockey()...comp
running agg_trainer()...
running agg_trainer()...comp


agg_horse_per_course_len:   0%|          | 0/5 [00:00<?, ?it/s]

running cross_features()...comp
running agg_interval()...comp
running cross_features_2()...comp


agg_horse_per_ground_state_race_type:   0%|          | 0/5 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/5 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/5 [00:00<?, ?it/s]

agg_horse_per_race_place_len:   0%|          | 0/5 [00:00<?, ?it/s]

agg_horse_per_weather:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_sire()...
running agg_sire()...comp
merging all features...
merging all features...comp


In [12]:
features

race_id        date    horse_id  jockey_id  trainer_id  owner_id  \
0      202306050101  2023-12-02  2021104145       1170        1052    266031   
1      202306050101  2023-12-02  2021104145       1170        1052    266031   
2      202306050101  2023-12-02  2021104145       1170        1052    266031   
3      202306050101  2023-12-02  2021104145       1170        1052    266031   
4      202306050101  2023-12-02  2021104145       1170        1052    266031   
...             ...         ...         ...        ...         ...       ...   
55359  202309050912  2023-12-28  2017102603       1144        1050    356002   
55360  202309050912  2023-12-28  2017102603       1144        1050    356002   
55361  202309050912  2023-12-28  2017102603       1144        1050    356002   
55362  202309050912  2023-12-28  2017102603       1144        1050    356002   
55363  202309050912  2023-12-28  2017102603       1144        1050    356002   

       rank  rank_per_horse  time  nobori  umaban  wakuban  tansho_odds  \
0         1          0.0625  72.8    37.8      15        8          1.9   
1         1          0.0625  72.8    37.8      15        8          1.9   
2         1          0.0625  72.8    37.8      15        8          1.9   
3         1          0.0625  72.8    37.8      15        8          1.9   
4         1          0.0625  72.8    37.8      15        8          1.9   
...     ...             ...   ...     ...     ...      ...          ...   
55359    16          1.0000  70.4    34.4      14        7        347.8   
55360    16          1.0000  70.4    34.4      14        7        347.8   
55361    16          1.0000  70.4    34.4      14        7        347.8   
55362    16          1.0000  70.4    34.4      14        7        347.8   
55363    16          1.0000  70.4    34.4      14        7        347.8   

       popularity  impost  sex  age  weight  weight_diff  n_horses  \
0               1    56.0    0    2     472           10        16   
1               1    56.0    0    2     472           10        16   
2               1    56.0    0    2     472           10        16   
3               1    56.0    0    2     472           10        16   
4               1    56.0    0    2     472           10        16   
...           ...     ...  ...  ...     ...          ...       ...   
55359          14    58.0    0    6     490           14        16   
55360          14    58.0    0    6     490           14        16   
55361          14    58.0    0    6     490           14        16   
55362          14    58.0    0    6     490           14        16   
55363          14    58.0    0    6     490           14        16   

       corner_1_per_horse  corner_2_per_horse  corner_3_per_horse  \
0                     0.5              0.5000                 NaN   
1                     0.5              0.5000                 NaN   
2                     0.5              0.5000                 NaN   
3                     0.5              0.5000                 NaN   
4                     0.5              0.5000                 NaN   
...                   ...                 ...                 ...   
55359                 1.0              0.9375                 NaN   
55360                 1.0              0.9375                 NaN   
55361                 1.0              0.9375                 NaN   
55362                 1.0              0.9375                 NaN   
55363                 1.0              0.9375                 NaN   

       corner_4_per_horse  corner_1  corner_2  corner_3  corner_4  race_type  \
0                     NaN         8         8       NaN       NaN          0   
1                     NaN         8         8       NaN       NaN          0   
2                     NaN         8         8       NaN       NaN          0   
3                     NaN         8         8       NaN       NaN          0   
4                     NaN         8         8       NaN       NaN          0   
